In [1]:
import datetime
import findspark
findspark.init()

In [2]:
from pyspark.sql.functions import rand,when,round,expr

# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('versioncontrol').getOrCreate()
import pyspark

#from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [3]:
data = [(1, 10, 5), (2, 15, 8), (3, 20, 12)]
df = spark.createDataFrame(data, ["id", "col1", "col2"])
df.show()

+---+----+----+
| id|col1|col2|
+---+----+----+
|  1|  10|   5|
|  2|  15|   8|
|  3|  20|  12|
+---+----+----+



In [6]:
rules = [
    {
        'function_name': 'function1',
        'start_date': datetime.date(2023, 1, 1),
        'end_date': datetime.date(2023, 6, 30),
        'code': """col1 * 3 * col2"""
    },
    {
        'function_name': 'function2',
        'start_date': datetime.date(2023, 7, 1),
        'end_date': datetime.date(2023, 7, 30),
        'code': """col1 * 2 * col2"""
        
    },
    # Add more rules as needed
]
#current_date = datetime.date.today()
date1 = datetime.date(2021, 4, 10)

# Add 1 to the date object


def foo():
    return "col1 * 2 * col2"

start_date = datetime.date(2023, 1, 1)
end_date = start_date + datetime.timedelta(days=365 * 4)  # Add 4 years to the start date

current_date = start_date

while current_date <= end_date:
    
    # Add 1 month to the current date
    year = current_date.year + ((current_date.month + 1) // 12)
    month = (current_date.month + 1) % 12
    if month == 0:
        month = 12
    current_date = current_date.replace(year=year, month=month)
    print("current_date",current_date)
    for rule in rules:
        if rule['start_date'] <= current_date <= rule['end_date']:
            # Execute the code for the matched rule
            print(rule['start_date'],rule['end_date'],rule['code'])
            result_df = df.select(expr(rule['code']).alias("calculation_result"))

            # Show the resulting DataFrame
            result_df.show()









current_date 2023-02-01
2023-01-01 2023-06-30 col1 * 3 * col2
+------------------+
|calculation_result|
+------------------+
|               150|
|               360|
|               720|
+------------------+

current_date 2023-03-01
2023-01-01 2023-06-30 col1 * 3 * col2
+------------------+
|calculation_result|
+------------------+
|               150|
|               360|
|               720|
+------------------+

current_date 2023-04-01
2023-01-01 2023-06-30 col1 * 3 * col2
+------------------+
|calculation_result|
+------------------+
|               150|
|               360|
|               720|
+------------------+

current_date 2023-05-01
2023-01-01 2023-06-30 col1 * 3 * col2
+------------------+
|calculation_result|
+------------------+
|               150|
|               360|
|               720|
+------------------+

current_date 2023-06-01
2023-01-01 2023-06-30 col1 * 3 * col2
+------------------+
|calculation_result|
+------------------+
|               150|
|             

In [ ]:


#spark = configure_spark_with_delta_pip(builder).getOrCreate()



In [7]:
def create_version1():
    # Data
    data = [(2001,"BAU","Java", "20000"), (2001,"BAU","Python", "100000"), (2001,"BAU","Scala", "3000")]

    # Columns
    columns = ["Year","DataType","language","users_count"]

    # Create DataFrame
    df = spark.createDataFrame(data).toDF(*columns)

    # Print DataFrame
    df.show()
    df.write.mode("overwrite").parquet("data/programming/key=1")
    
def create_version2():
    data = [(2002,"BAU","Scala", "2000",1), (2002,"BAU","Python", "200000",2), (2002,"BAU","Rust", "2500",4)]

    # Columns
    columns = ["Year","DataType","language","users_count","avg_rating"]

    # Create DataFrame
    df = spark.createDataFrame(data).toDF(*columns)


    df.show()
    df.write.mode("overwrite").parquet("data/programming/key=2")
    
def create_version3():
    data = [(2003,"BAU","Scala", "2000",1,1000), (2001,"RESTATE","Python", "101000",2,30030),(2003,"BAU","Python", "300000",2,30030), (2003,"RESTATE","Rust", "2500",4,3023)]

    # Columns
    columns = ["Year","DataType","language","users_count","avg_rating","total_queries"]

    # Create DataFrame
    df = spark.createDataFrame(data).toDF(*columns)


    df.show()
    df.write.mode("overwrite").parquet("data/programming/key=3")    

In [9]:
create_version1()
create_version2()
create_version3()

+----+--------+--------+-----------+
|Year|DataType|language|users_count|
+----+--------+--------+-----------+
|2001|     BAU|    Java|      20000|
|2001|     BAU|  Python|     100000|
|2001|     BAU|   Scala|       3000|
+----+--------+--------+-----------+

+----+--------+--------+-----------+----------+
|Year|DataType|language|users_count|avg_rating|
+----+--------+--------+-----------+----------+
|2002|     BAU|   Scala|       2000|         1|
|2002|     BAU|  Python|     200000|         2|
|2002|     BAU|    Rust|       2500|         4|
+----+--------+--------+-----------+----------+

+----+--------+--------+-----------+----------+-------------+
|Year|DataType|language|users_count|avg_rating|total_queries|
+----+--------+--------+-----------+----------+-------------+
|2003|     BAU|   Scala|       2000|         1|         1000|
|2001| RESTATE|  Python|     101000|         2|        30030|
|2003|     BAU|  Python|     300000|         2|        30030|
|2003| RESTATE|    Rust|       

In [10]:
df = spark.read.parquet("data/programming")

In [11]:
df.show()

+----+--------+--------+-----------+---+
|Year|DataType|language|users_count|key|
+----+--------+--------+-----------+---+
|2001| RESTATE|  Python|     101000|  3|
|2003|     BAU|  Python|     300000|  3|
|2003| RESTATE|    Rust|       2500|  3|
|2003|     BAU|   Scala|       2000|  3|
|2002|     BAU|  Python|     200000|  2|
|2002|     BAU|   Scala|       2000|  2|
|2002|     BAU|    Rust|       2500|  2|
|2001|     BAU|  Python|     100000|  1|
|2001|     BAU|    Java|      20000|  1|
|2001|     BAU|   Scala|       3000|  1|
+----+--------+--------+-----------+---+



In [12]:
df_merged1 = spark.read.parquet("data/programming/key=1")
df_merged1.show()

+----+--------+--------+-----------+
|Year|DataType|language|users_count|
+----+--------+--------+-----------+
|2001|     BAU|  Python|     100000|
|2001|     BAU|    Java|      20000|
|2001|     BAU|   Scala|       3000|
+----+--------+--------+-----------+



In [13]:
df_merged2 = spark.read.parquet("data/programming/key=2")

In [14]:
df_merged2.show()

+----+--------+--------+-----------+----------+
|Year|DataType|language|users_count|avg_rating|
+----+--------+--------+-----------+----------+
|2002|     BAU|  Python|     200000|         2|
|2002|     BAU|   Scala|       2000|         1|
|2002|     BAU|    Rust|       2500|         4|
+----+--------+--------+-----------+----------+



In [15]:
df_merged3 = spark.read.parquet("data/programming/key=3")

In [16]:
df_merged3.show()

+----+--------+--------+-----------+----------+-------------+
|Year|DataType|language|users_count|avg_rating|total_queries|
+----+--------+--------+-----------+----------+-------------+
|2001| RESTATE|  Python|     101000|         2|        30030|
|2003|     BAU|  Python|     300000|         2|        30030|
|2003| RESTATE|    Rust|       2500|         4|         3023|
|2003|     BAU|   Scala|       2000|         1|         1000|
+----+--------+--------+-----------+----------+-------------+



In [17]:
df_merged_all = spark.read.option("mergeSchema", "true").parquet("data/programming")

In [18]:
df_merged_all.show()

+----+--------+--------+-----------+----------+-------------+---+
|Year|DataType|language|users_count|avg_rating|total_queries|key|
+----+--------+--------+-----------+----------+-------------+---+
|2001| RESTATE|  Python|     101000|         2|        30030|  3|
|2003|     BAU|  Python|     300000|         2|        30030|  3|
|2003| RESTATE|    Rust|       2500|         4|         3023|  3|
|2003|     BAU|   Scala|       2000|         1|         1000|  3|
|2002|     BAU|  Python|     200000|         2|         null|  2|
|2002|     BAU|   Scala|       2000|         1|         null|  2|
|2002|     BAU|    Rust|       2500|         4|         null|  2|
|2001|     BAU|  Python|     100000|      null|         null|  1|
|2001|     BAU|    Java|      20000|      null|         null|  1|
|2001|     BAU|   Scala|       3000|      null|         null|  1|
+----+--------+--------+-----------+----------+-------------+---+

